# DDL de la base de datos **ClassicModels** en formato compatible con **PostgreSQL**:

https://relational-data.org/dataset/ClassicModels

```sql
-- Creación de la base de datos
CREATE DATABASE ClassicModels;
\c ClassicModels;

-- Creación de tablas
CREATE TABLE offices (
    officeCode VARCHAR(10) PRIMARY KEY,
    city VARCHAR(50) NOT NULL,
    phone VARCHAR(20) NOT NULL,
    addressLine1 VARCHAR(50) NOT NULL,
    addressLine2 VARCHAR(50),
    state VARCHAR(50),
    country VARCHAR(50) NOT NULL,
    postalCode VARCHAR(15) NOT NULL,
    territory VARCHAR(10) NOT NULL
);

CREATE TABLE employees (
    employeeNumber SERIAL PRIMARY KEY,
    lastName VARCHAR(50) NOT NULL,
    firstName VARCHAR(50) NOT NULL,
    extension VARCHAR(10) NOT NULL,
    email VARCHAR(100) NOT NULL,
    officeCode VARCHAR(10) NOT NULL REFERENCES offices(officeCode) ON DELETE CASCADE,
    reportsTo INT REFERENCES employees(employeeNumber) ON DELETE SET NULL,
    jobTitle VARCHAR(50) NOT NULL
);

CREATE TABLE customers (
    customerNumber SERIAL PRIMARY KEY,
    customerName VARCHAR(50) NOT NULL,
    contactLastName VARCHAR(50) NOT NULL,
    contactFirstName VARCHAR(50) NOT NULL,
    phone VARCHAR(20) NOT NULL,
    addressLine1 VARCHAR(50) NOT NULL,
    addressLine2 VARCHAR(50),
    city VARCHAR(50) NOT NULL,
    state VARCHAR(50),
    postalCode VARCHAR(15),
    country VARCHAR(50) NOT NULL,
    salesRepEmployeeNumber INT REFERENCES employees(employeeNumber) ON DELETE SET NULL,
    creditLimit DECIMAL(10, 2)
);

CREATE TABLE productLines (
    productLine VARCHAR(50) PRIMARY KEY,
    textDescription TEXT,
    htmlDescription TEXT,
    image BYTEA
);

CREATE TABLE products (
    productCode VARCHAR(15) PRIMARY KEY,
    productName VARCHAR(70) NOT NULL,
    productLine VARCHAR(50) NOT NULL REFERENCES productLines(productLine) ON DELETE CASCADE,
    productScale VARCHAR(10) NOT NULL,
    productVendor VARCHAR(50) NOT NULL,
    productDescription TEXT NOT NULL,
    quantityInStock INT NOT NULL,
    buyPrice DECIMAL(10, 2) NOT NULL,
    MSRP DECIMAL(10, 2) NOT NULL
);

CREATE TABLE orders (
    orderNumber SERIAL PRIMARY KEY,
    orderDate DATE NOT NULL,
    requiredDate DATE NOT NULL,
    shippedDate DATE,
    status VARCHAR(15) NOT NULL,
    comments TEXT,
    customerNumber INT NOT NULL REFERENCES customers(customerNumber) ON DELETE CASCADE
);

CREATE TABLE orderDetails (
    orderNumber INT NOT NULL REFERENCES orders(orderNumber) ON DELETE CASCADE,
    productCode VARCHAR(15) NOT NULL REFERENCES products(productCode) ON DELETE CASCADE,
    quantityOrdered INT NOT NULL,
    priceEach DECIMAL(10, 2) NOT NULL,
    orderLineNumber SMALLINT NOT NULL,
    PRIMARY KEY (orderNumber, productCode)
);

CREATE TABLE payments (
    customerNumber INT NOT NULL REFERENCES customers(customerNumber) ON DELETE CASCADE,
    checkNumber VARCHAR(50) NOT NULL,
    paymentDate DATE NOT NULL,
    amount DECIMAL(10, 2) NOT NULL,
    PRIMARY KEY (customerNumber, checkNumber)
);
```

### Notas sobre el DDL:
1. **Tipos de datos adaptados a PostgreSQL:**
   - `VARCHAR` en lugar de `CHAR` para cadenas de longitud variable.
   - `SERIAL` para las claves primarias automáticas (`AUTO_INCREMENT` en MySQL).
   - `BYTEA` para datos binarios como imágenes.
   - `DECIMAL(10, 2)` para valores monetarios.
   
2. **Restricciones y referencias:**
   - Se añadieron claves foráneas con reglas de eliminación apropiadas (`ON DELETE CASCADE`, `ON DELETE SET NULL`).
   - Las referencias circulares (como en `employees.reportsTo`) están permitidas con `NULL`.

3. **Compatibilidad:**
   - La tabla `orderDetails` usa una clave primaria compuesta, que es común en PostgreSQL.
   - Para manejar comentarios HTML o imágenes, se utilizan tipos como `TEXT` y `BYTEA`.
